<h1>
    Susceptability Artifact Correction without Fieldmaps
</h1>
<p style="font-size:10px;">12/09/2019, Andrew Van</p>

<p>This subslide runs a bunch of scripts for this presentation.</p>
<script>window.addEventListener('load', () => { Reveal.configure({ transition: 'fade' }); });</script>

- Why?
- Results
- How this works
- Some future directions

<h1>
    Susceptibility Artifact Correction
</h1>

- Caused by the interplay between the long readout time for BOLD imaging and the inhomogenities in the main magnetic field
    - Causes geometric distortions
- Usually corrected for by aquiring fieldmaps
    - Some problems: Extant datasets without fieldmaps/corrupted fieldmaps
- Correction without fieldmaps?
    - T1/T2 not affected by SA... Can we use this information for unwarping?

<img src="images/SAC.png" style="width:300px;height:auto;margin-left:auto;margin-right:auto;display:block;">

<h1>
    Fake functionals
</h1>

- Use T1/T2 to create a "fake" high-resolution functional using a transformation model
    - Align this to a reference functional
    - Requires estimating alignment and transformation model parameters at same time
        - Joint optimization required!
        
<img src="images/fake_func.png" style="width:75%;height:auto;margin-left:auto;margin-right:auto;display:block;">
<p style="text-align:center;">"Fake" functional derived from T1/T2</p>

<h1>
    Results (Shift maps, alignments, etc.)
</h1>
<p>Open External Viewer</p>

<h1>
    Algorithm summary
</h1>

With non-linear warp parameters $\gamma$:
$$
    y = f(M_{\theta}F\phi; \gamma) + n
$$

- Anatomical mask with skull
- Create reference volume of low movement frames
- Create mask of voxels with relatively low motion
- Do initial functional/anatomical alignement with MI metric
- Iterative Registration (3x):
    - Run joint optimization to get registration between anatomicals and reference functional***
    - Run 3dQwarp with low motion voxel mask

<h1>
    Registration Model
</h1>
<br>
$$
    y = M_{\theta}F\phi + n
$$

- $y \in \mathbb{R}^{m}$: distorted EPI image
- $F \in \mathbb{R}^{n \times w}$: basis spanned by T1/T2 kernels with rbf (T1/T2 image of size $n$)
    - i.e. $rbf(x;k) = \exp \left(\frac{x - c_{0}}{2\alpha^{2}}\right) + ... + \exp\left(\frac{x - c_{k}}{2\alpha^{2}}\right)$ where $c_{k}$ and $\alpha$ are set to cover the entire image histogram equidistantly
- $\phi \in \mathbb{R}^{w}$: weights of transform model
- $M_{\theta} \in \mathbb{R}^{m \times n}$: interpolation matrix parameterized by 12 parameter affine transform ($\theta$)
- $n$: additive gaussian noise

<h1>
    Joint optimization with Maximum Likelihood
</h1>
<br>
$$
    y = M_{\theta}F\phi + n
$$

- Objective: $\{\theta,\phi\} = \underset{\theta,\phi}{\arg \max}\ p(y|\phi,\theta)$
$$
    p(y|\phi,\theta) = \frac{1}{Z}\exp\left(\frac{-1}{2\sigma^{2}}\lVert y - M_{\theta}F\phi \rVert_{2}^{2}\right)
$$
where $Z$ is a normalizing factor and $\sigma$ is the std. dev. of the noise distribution

- Take the $-\log p(y|\phi,\theta)$
$$
    \underset{\theta,\phi}{\arg \min}\ \lVert y - M_{\theta}F\phi \rVert_{2}^{2}
$$


<h1>
    Alternating minimization
</h1>
<div style="margin-top:2rem;color:#FFF;">
    <div>1.) Initialize: $\theta_{0}$</div>
    <div>2.) for $j = 1,2,... k$ do</div>
    <div>3.) &nbsp;&nbsp;&nbsp; $\phi_{j} = (F^{\intercal}F)^{-1}F^{\intercal}M_{\theta_{j-1}}^{\intercal}y$</div>
    <div>4.) &nbsp;&nbsp;&nbsp; $\theta_{j} = \theta_{j-1} + \beta\nabla_{\theta}(\lVert y - M_{\theta}F\phi_{j} \rVert_{2}^{2})$</div>
    <div>5.) end for</div>
    <div>6.) return $\{\theta_{k},\phi_{k}\}$</div>
</div>

- Caveats:
    - Convergence to optimum only if $\theta_{0}$ is close!
    - Only affine transform (non-linear warps done by 3dQwarp)

<h1>
    Future directions
</h1>

- Comparison with other correction methods
    - Comparing to AFNI/FSL fieldmap correction methods
    - Compare to ANTs fieldmapless correction
    - Quantitative comparison? (Maybe look at intra/inter-subject variability on correlations)
- More efficient optimization using expectation-maximization instead of alternating minimization
- Incorporate own nonlinear registation into joint optimization algorithm